## I did not write this notebook. I just used it to train StyleGan2-ADA models. I followed the following youtube tutorials: https://www.youtube.com/watch?v=DVXX0tmVyco


In [ ]:
!nvidia-smi

## Install Repo to Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/gaetanlop/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

## Convert dataset to .tfrecords

In [ ]:
from PIL import Image
import os, sys

path = "/content/drive/My Drive/Poke_style/"
dirs = os.listdir( path )

def resize():
    f=0
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            im=im.convert("RGBA")
            background=Image.new(im.mode[:-1], im.size, (255,255,255))
            background.paste(im, im.split()[-1])
            im = background
            imResize = im.resize((128,128), Image.ANTIALIAS)
            imResize.save(str(f) + '_new_128.jpg', 'JPEG', quality=90)
            f+=1

In [ ]:
%cd "/content/drive/My Drive/"
!mkdir Poke_128

In [ ]:
%cd "/content/drive/My Drive/Poke_128"
resize()

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
#update this to the path to your image folder
dataset_path = "/content/drive/MyDrive/Poke_128"
#give your dataset a name
dataset_name = "Poke_128"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

## Train a custom model


In [ ]:
!python train.py --help

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "Poke_128"
#how often should the model generate samples and a .pkl file
snapshot_count = 6
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bgc"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00009-Poke_128-mirror-11gb-gpu-bgc-resumecustom/network-snapshot-000672.pkl"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --metrics=None --resume={resume_from}

# Usual Training

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "full_poke_128"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bgc"


#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00007-Humanoides-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000304.pkl"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --resume={resume_from} --metrics=None